In [ ]:
%load_ext autoreload
%autoreload 2
import panorama
import ransac_wikipedia

In [ ]:
%autoreload 2

panorama.main('./', 'cuda', ransac_reproj_thresh=5.0, ransac_max_iter=300, mode='plot')

In [ ]:
%autoreload 2
ransac_wikipedia.main()